# 네이버 영화평 감성 분석

In [1]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

- 데이터 가져오기: '네이버 영화리뷰 데이터'로 검색

In [2]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [3]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리
- 트레인 데이터 셋

In [4]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [5]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [6]:
# Null 데이터가 있는지 확인
train_df.document.isna().sum()

1

In [7]:
# Null 데이터 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [8]:
# 부정, 긍정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [9]:
# 중복 여부 확인
test_df.document.nunique()

49157

In [10]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [11]:
# Null 데이터가 있는지 확인
test_df.document.isna().sum()

1

In [12]:
# Null 데이터 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

### 2. 텍스트 전처리
- Train dataset

In [14]:
# 한글이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
# 데이터가 다 지워져서 ''가 된 데이터 --> np.nan으로 변경한 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.isna().sum()

id            0
document    789
label         0
dtype: int64

In [17]:
train_df = train_df.dropna(how='any')
train_df.shape

(145393, 3)

- Test dataset

In [18]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape

(48852, 3)

- 전처리가 끝난 데이터는 저장

In [19]:
train_df.to_csv('data/naver_movie_train.tsv', sep='\t', index=False)
test_df.to_csv('data/naver_movie_test.tsv', sep='\t', index=False)

### 3. 한글 처리 - Soynlp

In [20]:
import joblib
scores = joblib.load('data/scores.pkl')

In [21]:
from soynlp.tokenizer import MaxScoreTokenizer
max_tok = MaxScoreTokenizer(scores=scores)

In [24]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
max_tok.tokenize(text)

['교도소', '이야기', '구먼', '솔직', '히', '재미는', '없다', '평점', '조정']

In [25]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()

In [26]:
from tqdm.notebook import tqdm
X_train = []
for sentence in tqdm(train_df.document):
    words = max_tok.tokenize(sentence)
    tmp_list = [word for word in words if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_train.append(tmp_str)

In [27]:
X_test = []
for sentence in tqdm(test_df.document):
    words = max_tok.tokenize(sentence)
    tmp_list = [word for word in words if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_test.append(tmp_str)

In [28]:
y_train = train_df.label.values 
y_test = test_df.label.values 

### 4. Feature 변환

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((145393, 159491), (48852, 159491))

### 5. 모델 학습/평가

In [30]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)
lr.score(X_test_cv, y_test)

0.8315524441169246

### 6. 실제 테스트

In [31]:
review1 = '두시간 반이 왜 이렇게 짧냐'
review2 = '저 정말 스파이더맨 팬입니다. 정말 역겨움을 참느라 힘들었습니다. 이 영화가 재미있다고 하는 사람들 이해할 수가 없네요.'

In [32]:
# 전처리
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',review1).strip()
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',review2).strip()
review2

'저 정말 스파이더맨 팬입니다  정말 역겨움을 참느라 힘들었습니다  이 영화가 재미있다고 하는 사람들 이해할 수가 없네요'

In [33]:
# 불용어 처리
words = max_tok.tokenize(review1)
review1 = ' '.join(word for word in words if word not in stopwords)
review1

'두 시간 반이 왜 이렇게 짧냐'

In [34]:
review2 = ' '.join(word for word in max_tok.tokenize(review2) if word not in stopwords)
review2

'저 정말 스파이더맨 팬 입니다 정말 역겨움을 참느라 힘들 었습니다 영화 재미 있다 고 하는 사람들 이해할 수가 없네요'

In [35]:
# Feature 변환
review1_cv = cvect.transform([review1])
lr.predict(review1_cv)

array([0], dtype=int64)

In [36]:
review2_cv = cvect.transform([review2])
lr.predict(review2_cv)

array([1], dtype=int64)

### 7. GridSearchCV로 최적 파라미터 찾기

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [38]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lr', LogisticRegression())
])
params = {
    'cvect__ngram_range': [(1,1),(1,2)],
    'cvect__max_df': [0.9, 0.99],
    'cvect__min_df': [1, 3],
    'lr__C': [1, 5]
}

In [39]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3, scoring='accuracy', n_jobs=-1
)
%time grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_score_, grid_pipe.best_params_)

Wall time: 4min 9s
0.8329355414481997 {'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2), 'lr__C': 1}


In [40]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8407639400638663